# Input

In [ ]:
DO_SUBMISSION = True
DO_TRAIN_FOR_ENSEMBLE = False
DO_VIRTUAL_SUBMISSION = False
assert (sum([DO_SUBMISSION, DO_TRAIN_FOR_ENSEMBLE, DO_VIRTUAL_SUBMISSION]) == 1), "select `ONE` mode"

In [ ]:
import os
import gc
import sys
import time
import shutil
from pathlib import Path
from copy import deepcopy
from contextlib import contextmanager

import random
import typing as tp
from collections import OrderedDict


import yaml
import numpy as np
import pandas as pd

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [ ]:
@contextmanager
def timer(logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None):
    if prefix: format_str = str(prefix) + format_str
    if suffix: format_str = format_str + str(suffix)
    start = time.time()
    yield
    d = time.time() - start
    out_str = format_str.format(d)
    if logger:
        logger.info(out_str)
    else:
        print(out_str)

In [ ]:
def generate_virtual_private(test_feat, smpl_sub, rate_to_pub=4):
    
    assert (test_feat.sig_id == smpl_sub.sig_id).all()
    
    # public は含まれるので、そのまま入れる.
    test_feat_list = [test_feat]
    smpl_sub_list = [smpl_sub]
    
    for i in range(1, rate_to_pub):
        tmp_test_feat = test_feat.copy()
        tmp_smpl_sub = smpl_sub.copy()
        
        # #  id を変更
        tmp_test_feat.sig_id = tmp_test_feat.sig_id + f"_{i}"
        tmp_smpl_sub.sig_id = tmp_smpl_sub.sig_id + f"_{i}"
        assert (tmp_test_feat.sig_id == tmp_smpl_sub.sig_id).all()
        
        # # `c-*` と `g-*` に適当な値を加える. 複製して増やすだけだとすり抜ける場合があったため.
        tmp_test_feat.iloc[:, 4:] += i * 10
        assert (tmp_test_feat.iloc[:, 4:] != test_feat.iloc[:, 4:]).all().all()
        
        test_feat_list.append(tmp_test_feat)
        smpl_sub_list.append(tmp_smpl_sub)
        
    # # 結合
    test_feat_concat = pd.concat(test_feat_list, axis=0, ignore_index=True)
    smpl_sub_concat = pd.concat(smpl_sub_list, axis=0, ignore_index=True)
    
    return test_feat_concat, smpl_sub_concat

# pip install

In [ ]:
%%time
# for nn.py
!pip install ../input/iterative-stratification/iterative-stratification-master/

In [ ]:
%%time
# for tabnet
!pip install --no-index --find-links ../input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

# Inference Test by Stage1 Models

In [ ]:
%%time
if DO_SUBMISSION:
    !python ../input/moa-nn-tabnet-fix5/nn-inference-0.01833.py

elif DO_VIRTUAL_SUBMISSION:
    !python ../input/moa-nn-tabnet-fix5/virtual-nn-use-train-public-inference.py

elif DO_TRAIN_FOR_ENSEMBLE:
    pass
else:
    raise ValueError

In [ ]:
%%time
if DO_SUBMISSION:
    !python ../input/moa-nn-tabnet-fix5/tabnet-inference-0.01840.py

elif DO_VIRTUAL_SUBMISSION:
    !python ../input/moa-nn-tabnet-fix5/virtual-tabnet-inference-add-param-n-shared.py

elif DO_TRAIN_FOR_ENSEMBLE:
    pass
else:
    raise ValueError

In [ ]:
%%time
if DO_SUBMISSION:
    !python ../input/moa-takapy-script/tf-rn-transfer-1layerother-selcol100.py  # 0.01862

elif DO_VIRTUAL_SUBMISSION:
    !python ../input/moa-takapy-script/virtual-tf-rn-transfer-1layerother-selcol100.py

elif DO_TRAIN_FOR_ENSEMBLE:
    pass
else:
    raise ValueError

In [ ]:
%%time
if DO_SUBMISSION:
    !python ../input/moa-tawara-scripts-for-final-submission/moa-for-final-thrnn-seed-cv-0.01836.py

elif DO_VIRTUAL_SUBMISSION:
    !python ../input/moa-tawara-scripts-for-final-submission/virtual-moa-for-final-thrnn-seed-cv-0.01836.py

elif DO_TRAIN_FOR_ENSEMBLE:
    pass
else:
    raise ValueError

In [ ]:
%%time
if DO_SUBMISSION:
    !python ../input/moa-tawara-scripts-for-final-submission/moa-for-final-thrnn-drug-seed-cv-0.01841.py

elif DO_VIRTUAL_SUBMISSION:
    !python ../input/moa-tawara-scripts-for-final-submission/virtual-moa-for-final-thrnn-drug-seed-cv-0.01841.py

elif DO_TRAIN_FOR_ENSEMBLE:
    pass
else:
    raise ValueError

# Load OOF predcitions

In [ ]:
TARGET_COL = ['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor', 'acat_inhibitor', 'acetylcholine_receptor_agonist', 'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor', 'adenosine_receptor_agonist', 'adenosine_receptor_antagonist', 'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist', 'adrenergic_receptor_antagonist', 'akt_inhibitor', 'aldehyde_dehydrogenase_inhibitor', 'alk_inhibitor', 'ampk_activator', 'analgesic', 'androgen_receptor_agonist', 'androgen_receptor_antagonist', 'anesthetic_-_local', 'angiogenesis_inhibitor', 'angiotensin_receptor_antagonist', 'anti-inflammatory', 'antiarrhythmic', 'antibiotic', 'anticonvulsant', 'antifungal', 'antihistamine', 'antimalarial', 'antioxidant', 'antiprotozoal', 'antiviral', 'apoptosis_stimulant', 'aromatase_inhibitor', 'atm_kinase_inhibitor', 'atp-sensitive_potassium_channel_antagonist', 'atp_synthase_inhibitor', 'atpase_inhibitor', 'atr_kinase_inhibitor', 'aurora_kinase_inhibitor', 'autotaxin_inhibitor', 'bacterial_30s_ribosomal_subunit_inhibitor', 'bacterial_50s_ribosomal_subunit_inhibitor', 'bacterial_antifolate', 'bacterial_cell_wall_synthesis_inhibitor', 'bacterial_dna_gyrase_inhibitor', 'bacterial_dna_inhibitor', 'bacterial_membrane_integrity_inhibitor', 'bcl_inhibitor', 'bcr-abl_inhibitor', 'benzodiazepine_receptor_agonist', 'beta_amyloid_inhibitor', 'bromodomain_inhibitor', 'btk_inhibitor', 'calcineurin_inhibitor', 'calcium_channel_blocker', 'cannabinoid_receptor_agonist', 'cannabinoid_receptor_antagonist', 'carbonic_anhydrase_inhibitor', 'casein_kinase_inhibitor', 'caspase_activator', 'catechol_o_methyltransferase_inhibitor', 'cc_chemokine_receptor_antagonist', 'cck_receptor_antagonist', 'cdk_inhibitor', 'chelating_agent', 'chk_inhibitor', 'chloride_channel_blocker', 'cholesterol_inhibitor', 'cholinergic_receptor_antagonist', 'coagulation_factor_inhibitor', 'corticosteroid_agonist', 'cyclooxygenase_inhibitor', 'cytochrome_p450_inhibitor', 'dihydrofolate_reductase_inhibitor', 'dipeptidyl_peptidase_inhibitor', 'diuretic', 'dna_alkylating_agent', 'dna_inhibitor', 'dopamine_receptor_agonist', 'dopamine_receptor_antagonist', 'egfr_inhibitor', 'elastase_inhibitor', 'erbb2_inhibitor', 'estrogen_receptor_agonist', 'estrogen_receptor_antagonist', 'faah_inhibitor', 'farnesyltransferase_inhibitor', 'fatty_acid_receptor_agonist', 'fgfr_inhibitor', 'flt3_inhibitor', 'focal_adhesion_kinase_inhibitor', 'free_radical_scavenger', 'fungal_squalene_epoxidase_inhibitor', 'gaba_receptor_agonist', 'gaba_receptor_antagonist', 'gamma_secretase_inhibitor', 'glucocorticoid_receptor_agonist', 'glutamate_inhibitor', 'glutamate_receptor_agonist', 'glutamate_receptor_antagonist', 'gonadotropin_receptor_agonist', 'gsk_inhibitor', 'hcv_inhibitor', 'hdac_inhibitor', 'histamine_receptor_agonist', 'histamine_receptor_antagonist', 'histone_lysine_demethylase_inhibitor', 'histone_lysine_methyltransferase_inhibitor', 'hiv_inhibitor', 'hmgcr_inhibitor', 'hsp_inhibitor', 'igf-1_inhibitor', 'ikk_inhibitor', 'imidazoline_receptor_agonist', 'immunosuppressant', 'insulin_secretagogue', 'insulin_sensitizer', 'integrin_inhibitor', 'jak_inhibitor', 'kit_inhibitor', 'laxative', 'leukotriene_inhibitor', 'leukotriene_receptor_antagonist', 'lipase_inhibitor', 'lipoxygenase_inhibitor', 'lxr_agonist', 'mdm_inhibitor', 'mek_inhibitor', 'membrane_integrity_inhibitor', 'mineralocorticoid_receptor_antagonist', 'monoacylglycerol_lipase_inhibitor', 'monoamine_oxidase_inhibitor', 'monopolar_spindle_1_kinase_inhibitor', 'mtor_inhibitor', 'mucolytic_agent', 'neuropeptide_receptor_antagonist', 'nfkb_inhibitor', 'nicotinic_receptor_agonist', 'nitric_oxide_donor', 'nitric_oxide_production_inhibitor', 'nitric_oxide_synthase_inhibitor', 'norepinephrine_reuptake_inhibitor', 'nrf2_activator', 'opioid_receptor_agonist', 'opioid_receptor_antagonist', 'orexin_receptor_antagonist', 'p38_mapk_inhibitor', 'p-glycoprotein_inhibitor', 'parp_inhibitor', 'pdgfr_inhibitor', 'pdk_inhibitor', 'phosphodiesterase_inhibitor', 'phospholipase_inhibitor', 'pi3k_inhibitor', 'pkc_inhibitor', 'potassium_channel_activator', 'potassium_channel_antagonist', 'ppar_receptor_agonist', 'ppar_receptor_antagonist', 'progesterone_receptor_agonist', 'progesterone_receptor_antagonist', 'prostaglandin_inhibitor', 'prostanoid_receptor_antagonist', 'proteasome_inhibitor', 'protein_kinase_inhibitor', 'protein_phosphatase_inhibitor', 'protein_synthesis_inhibitor', 'protein_tyrosine_kinase_inhibitor', 'radiopaque_medium', 'raf_inhibitor', 'ras_gtpase_inhibitor', 'retinoid_receptor_agonist', 'retinoid_receptor_antagonist', 'rho_associated_kinase_inhibitor', 'ribonucleoside_reductase_inhibitor', 'rna_polymerase_inhibitor', 'serotonin_receptor_agonist', 'serotonin_receptor_antagonist', 'serotonin_reuptake_inhibitor', 'sigma_receptor_agonist', 'sigma_receptor_antagonist', 'smoothened_receptor_antagonist', 'sodium_channel_inhibitor', 'sphingosine_receptor_agonist', 'src_inhibitor', 'steroid', 'syk_inhibitor', 'tachykinin_antagonist', 'tgf-beta_receptor_inhibitor', 'thrombin_inhibitor', 'thymidylate_synthase_inhibitor', 'tlr_agonist', 'tlr_antagonist', 'tnf_inhibitor', 'topoisomerase_inhibitor', 'transient_receptor_potential_channel_antagonist', 'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist', 'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor', 'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b', 'vitamin_d_receptor_agonist', 'wnt_inhibitor']

In [ ]:
def del_control(df: pd.DataFrame()) -> pd.DataFrame():
    train = pd.read_csv('../input/lish-moa/train_features.csv')
    df['cp_type'] = train['cp_type']
    df = df.loc[df['cp_type']=='trt_cp'].reset_index(drop=True)
    df = df.drop('cp_type', axis=1)
    return df

def drop_sig_id(df: pd.DataFrame()) -> pd.DataFrame():
    return df.drop('sig_id', axis=1)

def fix_oof(df: pd.DataFrame()) -> pd.DataFrame():
    if df.shape[0] != 21948:
        df = del_control(df)
    if 'sig_id' in df.columns.tolist():
        df = drop_sig_id(df)
    return df

In [ ]:
oof_list = []

In [ ]:
# sinchir0
# nn
oof_list.append(fix_oof(
    pd.read_pickle('../input/nn-use-train-public/oof.pkl')))

# tabnet
oof_list.append(fix_oof(
    pd.DataFrame(
        np.load('../input/tabnet-train-public-add-n-shared-1/oof_tabnet.npy'),
        columns=TARGET_COL))
)

##################################
# asteriam


##################################
# takapy
oof_list.append(fix_oof(
    pd.read_csv("../input/moa-takapy-tf-resnet-transfer/oof_prediction_takapy_resnet.csv")))

##################################
# Tawara
# thrnn - seed cv
oof_list.append(fix_oof(
    pd.read_csv("../input/moa-weight-thrnn-seed-cv/oof_prediction.csv")))

# thrnn - drug seed cv
oof_list.append(fix_oof(
    pd.read_csv("../input/moa-weight-thrnn-drug-seed-cv/oof_prediction.csv")))

##################################
# Taro Masuda
# 

In [ ]:
y_true = fix_oof(pd.read_csv('../input/lish-moa/train_targets_scored.csv'))

In [ ]:
# confirm shape
print("sinchir0 - NN:", oof_list[0].shape)
print("local logloss:", log_loss(y_true.values.ravel(), oof_list[0].values.ravel()))
display(oof_list[0].head())

print("sinchir0 - TabNet:", oof_list[1].shape)
print("local logloss:", log_loss(y_true.values.ravel(), oof_list[1].values.ravel()))
display(oof_list[1].head())

print("takapy - ResNet:", oof_list[2].shape)
print("local logloss:", log_loss(y_true.values.ravel(), oof_list[2].values.ravel()))
display(oof_list[2].head())

print("tawara - ThrNN seed-cv:", oof_list[3].shape)
print("local logloss:", log_loss(y_true.values.ravel(), oof_list[3].values.ravel()))
display(oof_list[3].head())

print("tawara - ThrNN drug-seed-cv:", oof_list[4].shape)
print("local logloss:", log_loss(y_true.values.ravel(), oof_list[4].values.ravel()))
display(oof_list[4].head())

## Arithmetic mean

In [ ]:
Blend_oof = oof_list[0].copy()
Blend_oof[:] = 0.
for oof_df in oof_list:
    Blend_oof += oof_df
    
Blend_oof = Blend_oof / len(oof_list)

In [ ]:
y_true = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
y_true = fix_oof(y_true)

y_pred = Blend_oof.copy()

# score = 0
# for i in range(len(oof_list[0].columns.tolist())):
#     score_ = log_loss(y_true.iloc[:, i], y_pred.iloc[:, i])
#     score += score_ / oof_list[0].shape[1]

# print("CV log_loss: ", score)

print("CV log_loss: ", log_loss(y_true.values.ravel(), y_pred.values.ravel()))

# Load Output - Submission file

In [ ]:
if DO_SUBMISSION or DO_VIRTUAL_SUBMISSION:
    sub_list = [
        pd.read_csv('./submission-sinchir0-nn.csv'),
        pd.read_csv('./submission_sinchir0_tabnet.csv'),
        pd.read_csv("./submission_takapy_tf-resnet.csv"),
        pd.read_csv('./submission_tawara_thrnn_seed_cv.csv'),
        pd.read_csv('./submission_tawara_thrnn_drug_seed_cv.csv'),
    ]
else:
    sub_list = [
        pd.read_csv('../input/nn-use-train-public/submission.csv'),
        pd.read_csv('../input/tabnet-train-public-add-n-shared-1/submission.csv'),
        pd.read_csv("../input/moa-takapy-tf-resnet-transfer/submission.csv"),
        pd.read_csv('../input/moa-weight-thrnn-seed-cv/submission.csv'),
        pd.read_csv('../input/moa-weight-thrnn-drug-seed-cv/submission.csv'),
    ]

## Check Output

In [ ]:
# sinchir0 - NN model
print("shape:", sub_list[0].shape)
display(sub_list[0].head())

In [ ]:
# sinchir0 - TabNet model
print("shape:", sub_list[1].shape)
display(sub_list[1].head())

In [ ]:
# takapy - tf resnet
print("shape:", sub_list[2].shape)
display(sub_list[2].head())

In [ ]:
# tawara - ThrNN model (seed CV)
print("shape:", sub_list[3].shape)
display(sub_list[3].head())

In [ ]:
# tawara - ThrNN model (drug seed CV)
print("shape:", sub_list[4].shape)
display(sub_list[4].head())

## Magic Function for private

In [ ]:
def order_sub(sub) : 
    return sub.sort_values('sig_id').reset_index(drop=True)

In [ ]:
sub_list = [order_sub(sub_df) for sub_df in sub_list]

# Inference OOF and Test by Stacking Models

In [ ]:
import torch
from torch import nn
from torch.utils import data
torch.backends.cudnn.benchmark = True

os.environ["CUDA_VISIBLE_DEVICES"]="0"

sys.path.append("../input/iterative-stratification/iterative-stratification-master")
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

sys.path.append("../input/pytorch-pfn-extras/pytorch-pfn-extras-0.3.1/")
import pytorch_pfn_extras as ppe
from pytorch_pfn_extras.training import extensions as ppe_extensions

## prepare

### definition

In [ ]:
class MoAStackingDataset(data.Dataset):
    
    def __init__(self, feat: np.ndarray, label: np.ndarray = None):
        """"""
        self.feat = feat
        if label is None:
            self.label = np.full((len(feat), 1), -1)
        else:
            self.label = label
        
    def __len__(self):
        """"""
        return len(self.feat)
    
    def __getitem__(self, index: int):
        """"""
        return [
            torch.from_numpy(self.feat[index]).float(),
            torch.from_numpy(self.label[index]).float()
        ]


class MoAStackingDataset2d(data.Dataset):
    
    def __init__(self, feat: np.ndarray, label: np.ndarray = None):
        """"""
        self.feat = feat
        if label is None:
            self.label = np.full((len(feat), 1), -1)
        else:
            self.label = label
        self.reset_model_order()
        
    def reset_model_order(self):
        self.model_order = np.arange(self.feat.shape[-1])
        
    def shuffle_model_order(self, seed):
        np.random.seed(seed)
        self.model_order = np.random.permutation(self.model_order)
        
    def __len__(self):
        """"""
        return len(self.feat)
    
    def __getitem__(self, index: int):
        """"""
        return [
            torch.from_numpy(self.feat[index][:, self.model_order]).float(),
            torch.from_numpy(self.label[index]).float()
        ]

In [ ]:
def get_activation(activ_name: str="relu"):
    """"""
    act_dict = {
        "relu": nn.ReLU(),
        "tanh": nn.Tanh(),
        "sigmoid": nn.Sigmoid(),
        "identity": nn.Identity()}
    if activ_name in act_dict:
        return act_dict[activ_name]
    elif re.match(r"^htanh\_\d{4}$", activ_name):
        bound = int(activ_name[-4:]) / 1000
        return nn.Hardtanh(-bound, bound)
    else:
        raise NotImplementedError

class LBAD(nn.Module):
    """Linear (-> BN) -> Activation (-> Dropout)"""
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(LBAD, self).__init__()
        layers = [nn.Linear(in_features, out_features)]
        if use_wn:
            layers[0] = nn.utils.weight_norm(layers[0])
        
        if use_bn:
            layers.append(nn.BatchNorm1d(out_features))
        
        layers.append(get_activation(activ))
        
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
    
    
class BDLA(nn.Module):
    """(BN -> Dropout ->) Linear -> Activation"""
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(BDLA, self).__init__()
        layers = []
        if use_bn:
            layers.append(nn.BatchNorm1d(in_features))
            
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        layers.append(nn.Linear(in_features, out_features))
        if use_wn:
            layers[-1] = nn.utils.weight_norm(layers[-1])
            
        layers.append(get_activation(activ))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
    

class LABD(nn.Module):
    """Linear -> Activation (-> BN -> Dropout) """
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(LABD, self).__init__()
        layers = [nn.Linear(in_features, out_features), get_activation(activ)]
        
        if use_wn:
            layers[0] = nn.utils.weight_norm(layers[0])
        
        if use_bn:
            layers.append(nn.BatchNorm1d(out_features))
        
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)

In [ ]:
class MLP(nn.Module):
    """Stacked Dense layers"""
    
    def __init__(
        self, n_features_list: tp.List[int], use_tail_as_out: bool=False,
        drop_rate: float=0.0, use_bn: bool=False, use_wn: bool=False,
        activ:str="relu", block_name: str="LBAD"
    ):
        """"""
        super(MLP, self).__init__()
        n_layers = len(n_features_list) - 1
        block_class = {
            "LBAD": LBAD, "BDLA": BDLA, "LABD": LABD}[block_name]
        layers = []
        for i in range(n_layers):
            in_feats, out_feats = n_features_list[i: i + 2]
            if i == n_layers - 1 and use_tail_as_out:
                if block_name in ["BDLA"]:
                    layer = block_class(in_feats, out_feats, drop_rate, use_bn,  use_wn, "identity")
                else:
                    layer = nn.Linear(in_feats, out_feats)
                    if use_wn:
                        layer = nn.utils.weight_norm(layer)
            else:
                layer = block_class(in_feats, out_feats, drop_rate, use_bn,  use_wn, activ)
            layers.append(layer)
                
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)


class CNNForStacking(nn.Module):
    """"""
    
    def __init__(
        self, n_models: int,
        n_channels_list: tp.List[int],
        kwargs_head: tp.Dict,
    ):
        """"""
        super(CNNForStacking, self).__init__()
        self.n_conv_layers = len(n_channels_list) - 1
        for i in range(self.n_conv_layers):
            in_ch = n_channels_list[i]
            out_ch = n_channels_list[i + 1]
            layer = nn.Sequential(
                nn.Conv1d(in_ch, out_ch, 3, 1, 0),
                nn.BatchNorm1d(out_ch),
                nn.ReLU(inplace=True))
            setattr(self, "conv{}".format(i + 1), layer)
        
        kwargs_head["n_features_list"][0] = (n_models - 2 * self.n_conv_layers) * n_channels_list[-1]
        self.head = MLP(**kwargs_head)
    
    def forward(self, x: torch.FloatTensor) -> torch.Tensor:
        """"""
        bs = x.shape[0]
        h = x  # shape: (bs, n_classes, n_models)
        for i in range(self.n_conv_layers):
            h = getattr(self, "conv{}".format(i + 1))(h)
            
        h = torch.reshape(h, (bs, -1))
        h = self.head(h)
        return h

In [ ]:
def set_random_seed(seed: int = 42, deterministic: bool = False):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    if deterministic:
        torch.backends.cudnn.deterministic = True  # type: ignore

In [ ]:
def inference_function(settings, model, loader, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for x, t in loader:
            y = model(x.to(device))
            pred_list.append(y.sigmoid().detach().cpu().numpy())
        
        pred_arr = np.concatenate(pred_list)
        del pred_list
    return pred_arr

### create dataset

In [ ]:
X = np.concatenate([df.values for df in oof_list], axis=1)
print(X.shape)

y = y_true.values
print(y.shape)

train_all_dataset = MoAStackingDataset(X, y)

X_test = np.concatenate([
    sub_df.iloc[:, 1:].values for sub_df in sub_list], axis=1)
print(X_test.shape)
test_dataset = MoAStackingDataset(X_test, None)

X_2d = np.stack([df.values for df in oof_list], axis=2)
print(X_2d.shape)

train_all_dataset_2d = MoAStackingDataset2d(X_2d, y)

X_test_2d = np.stack(
    [sub_df.iloc[:, 1:].values for sub_df in sub_list], axis=2)
print(X_test_2d.shape)
test_dataset_2d = MoAStackingDataset2d(X_test_2d, None)

### MLP

In [ ]:
settings_str = """
globals:
  seed: 1990
  seeds_for_avg: [
    1990,
    42, 0, 1086, 39
  ]
  max_epoch: 20
  n_folds_split: 7
  patience: 10
  cuda_visible_devices: 0
  device: cuda
  fast_commit: false

loader:
  train: {
    batch_size: 128, shuffle: True, num_workers: 2,
    pin_memory: True, drop_last: True}
  val: {
    batch_size: 256, shuffle: False, num_workers: 2,
    pin_memory: True, drop_last: False}

model:
  name: MLP
  params:
    n_features_list: [-1, 1024, 1024, 206]
    use_tail_as_out: True
    drop_rate: 0.2
    use_bn: True
    use_wn: True
    block_name: BDLA
        
loss:
  name: MyLSLogLoss
  params: {k: 2, alpha: 1.0e-03}

optimizer:
  name: Adam
  # params: {lr: 1.0e-03}
  params: {lr: 1.0e-03, weight_decay: 5.0e-06}

scheduler:
  name: OneCycleLR
  params: {pct_start: 0.1, div_factor: 1.0e+3, max_lr: 1.0e-02}
"""

settings = yaml.safe_load(settings_str)
settings["model"]["params"]["n_features_list"][0] = len(oof_list) * 206

In [ ]:
device = torch.device(settings["globals"]["device"])
cv_score_list_mlp = []
stgs_list_mlp = []

# # seed avg
for tmp_seed in settings["globals"]["seeds_for_avg"]:
    stgs_list_mlp.append(deepcopy(settings))
    stgs_list_mlp[-1]["globals"]["seed"] = tmp_seed

In [ ]:
oof_pred_arr_avg_mlp = np.zeros((len(X), 206))
test_pred_arr_avg_mlp = np.zeros((len(X_test), 206))

for m_id, tmp_stgs in enumerate(stgs_list_mlp):
    n_folds = tmp_stgs["globals"]["n_folds_split"]
    tmp_seed = tmp_stgs["globals"]["seed"]
    mlskf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=tmp_seed)
    train_val_indexs = list(mlskf.split(train_all_dataset.feat, train_all_dataset.label))
    
    oof_pred_arr = np.zeros((len(X), 206))
    test_preds_arr = np.zeros((n_folds, len(X_test), 206))
    
    for fold_id in range(n_folds):
        print("[fold: {} - model: {}]".format(fold_id, m_id))
        model_path = "../input/moa-weight-stacking-5model-by-mlp-seed-cv/best_model_fold{}_model{}.pth".format(fold_id, m_id)
        model = MLP(**tmp_stgs["model"]["params"])
        model.load_state_dict(torch.load(model_path))

        val_index = train_val_indexs[fold_id][1]
        val_dataset = data.Subset(train_all_dataset, val_index)
        val_loader = data.DataLoader(val_dataset, **tmp_stgs["loader"]["val"])

        val_pred = inference_function(tmp_stgs, model, val_loader, device)
        oof_pred_arr[val_index] = val_pred

        del val_dataset; del val_loader; del val_pred

        test_loader = data.DataLoader(test_dataset, **tmp_stgs["loader"]["val"])
        test_pred = inference_function(tmp_stgs, model, test_loader, device)
        test_preds_arr[fold_id] = test_pred

        del test_loader; del test_pred;
        gc.collect()
    
    oof_pred_arr_avg_mlp += oof_pred_arr
    test_pred_arr_avg_mlp += test_preds_arr.mean(axis=0)
    
    oof_score = log_loss(y.ravel(), oof_pred_arr.ravel())

    cv_score_list_mlp.append([m_id, "oof", oof_score])
    
oof_pred_arr_avg_mlp /= len(stgs_list_mlp)
test_pred_arr_avg_mlp /= len(stgs_list_mlp)

In [ ]:
pd.DataFrame(cv_score_list_mlp, columns=["model", "fold", "val loss"])

In [ ]:
print(log_loss(y.ravel(), oof_pred_arr_avg_mlp.ravel()))

In [ ]:
BLEND_mlp = sub_list[0].copy()
BLEND_mlp.iloc[:, 1:] = test_pred_arr_avg_mlp

print("shape:", BLEND_mlp.shape)
display(BLEND_mlp.head())

### 1D-CNN

In [ ]:
settings_str = """
globals:
  seed: 1990
  seeds_for_avg: [
    1990,
    42, 0, 1086, 39
  ]
  max_epoch: 20
  n_folds_split: 7
  patience: 10
  cuda_visible_devices: 0
  device: cuda
  fast_commit: False

loader:
  train: {
    batch_size: 128, shuffle: True, num_workers: 2,
    pin_memory: True, drop_last: True}
  val: {
    batch_size: 256, shuffle: False, num_workers: 2,
    pin_memory: True, drop_last: False}

model:
  name: CNNforStacking
  params:
    n_models: -1
    n_channels_list: [206, 512, 1024]
    kwargs_head:
        n_features_list: [-1, 1024, 206]
        use_tail_as_out: True
        drop_rate: 0.2
        use_bn: True
        use_wn: True
        block_name: BDLA
        
loss:
  name: MyLSLogLoss
  params: {k: 2, alpha: 1.0e-03}

optimizer:
  name: Adam
  params: {lr: 1.0e-03, weight_decay: 5.0e-06}

scheduler:
  name: OneCycleLR
  params: {pct_start: 0.1, div_factor: 1.0e+3, max_lr: 1.0e-02}
"""

settings = yaml.safe_load(settings_str)

settings["model"]["params"]["n_models"] = len(oof_list)

In [ ]:
device = torch.device(settings["globals"]["device"])
cv_score_list_cnn = []

stgs_list_cnn = []

# # seed avg
for tmp_seed in settings["globals"]["seeds_for_avg"]:
    stgs_list_cnn.append(deepcopy(settings))
    stgs_list_cnn[-1]["globals"]["seed"] = tmp_seed

In [ ]:
oof_pred_arr_avg_cnn = np.zeros((len(X), 206))
test_pred_arr_avg_cnn = np.zeros((len(X_test), 206))

for m_id, tmp_stgs in enumerate(stgs_list_cnn):
    n_folds = tmp_stgs["globals"]["n_folds_split"]
    tmp_seed = tmp_stgs["globals"]["seed"]
    mlskf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=tmp_seed)
    train_val_indexs = list(mlskf.split(train_all_dataset.feat, train_all_dataset.label))
    
    oof_pred_arr = np.zeros((len(X), 206))
    test_preds_arr = np.zeros((n_folds, len(X_test), 206))
    
    train_all_dataset_2d.reset_model_order()
    train_all_dataset_2d.shuffle_model_order(tmp_seed)
    test_dataset_2d.model_order = train_all_dataset_2d.model_order
    print("[model {}'s order]: {}".format(m_id, train_all_dataset_2d.model_order))
    
    for fold_id in range(n_folds):
        print("[fold: {} - model: {}]".format(fold_id, m_id))
        model_path = "../input/moa-weight-stacking-5model-by-1dcnn-seed-cv/best_model_cnn_fold{}_model{}.pth".format(fold_id, m_id)
        model = CNNForStacking(**tmp_stgs["model"]["params"])
        model.load_state_dict(torch.load(model_path))

        val_index = train_val_indexs[fold_id][1]
        val_dataset = data.Subset(train_all_dataset_2d, val_index)
        val_loader = data.DataLoader(val_dataset, **tmp_stgs["loader"]["val"])

        val_pred = inference_function(tmp_stgs, model, val_loader, device)
        oof_pred_arr[val_index] = val_pred

        del val_dataset; del val_loader; del val_pred

        test_loader = data.DataLoader(test_dataset_2d, **tmp_stgs["loader"]["val"])
        test_pred = inference_function(tmp_stgs, model, test_loader, device)
        test_preds_arr[fold_id] = test_pred

        del test_loader; del test_pred;
        gc.collect()
    
    oof_pred_arr_avg_cnn += oof_pred_arr
    test_pred_arr_avg_cnn += test_preds_arr.mean(axis=0)
    
    oof_score = log_loss(y.ravel(), oof_pred_arr.ravel())
    cv_score_list_cnn.append([m_id, "oof",  oof_score])
    
oof_pred_arr_avg_cnn /= len(stgs_list_cnn)
test_pred_arr_avg_cnn /= len(stgs_list_cnn)

In [ ]:
pd.DataFrame(cv_score_list_cnn, columns=["model", "fold", "val loss"])

In [ ]:
print(log_loss(y.ravel(), oof_pred_arr_avg_cnn.ravel()))

In [ ]:
BLEND_cnn = sub_list[0].copy()
BLEND_cnn.iloc[:, 1:] = test_pred_arr_avg_cnn

print("shape:", BLEND_cnn.shape)
display(BLEND_cnn.head())

### WO

In [ ]:
opt_weights = [ 0.1709929 ,  0.22160569,  0.07207321,  0.95431566, -0.42108426]

In [ ]:
oof_pred_arr_avg_wo = np.zeros((len(X),206))

for w, oof_df in zip(opt_weights, oof_list):
    oof_pred_arr_avg_wo += oof_df.values * w

print(log_loss(y.ravel(), oof_pred_arr_avg_wo.ravel()))

In [ ]:
BLEND_wo = sub_list[0].copy()
BLEND_wo.iloc[:, 1:] = 0.

for w, sub_df in zip(opt_weights, sub_list):
    BLEND_wo.iloc[:,1:] += sub_df.iloc[:,1:] * w

## averaging stacking models

In [ ]:
oof_pred_arr_avg = (oof_pred_arr_avg_mlp + oof_pred_arr_avg_cnn + oof_pred_arr_avg_wo) / 3

print(log_loss(y.ravel(), oof_pred_arr_avg.ravel()))

In [ ]:
BLEND = sub_list[0].copy()
BLEND.iloc[:, 1:] = (BLEND_mlp.iloc[:, 1:] + BLEND_cnn.iloc[:, 1:] + BLEND_wo.iloc[:,1:]) / 3

BLEND.head()

## Make Submission

In [ ]:
if DO_SUBMISSION or DO_TRAIN_FOR_ENSEMBLE:
    df_test = pd.read_csv("../input/lish-moa/test_features.csv")
    submission = pd.read_csv("../input/lish-moa/sample_submission.csv")

elif DO_VIRTUAL_SUBMISSION:
    df_test = pd.read_csv("../input/lish-moa/test_features.csv")
    submission = pd.read_csv("../input/lish-moa/sample_submission.csv")
    print(df_test.shape, submission.shape)
    df_test, submission = generate_virtual_private(df_test, submission)
    print(df_test.shape, submission.shape)

else:
    raise ValueError

In [ ]:
# submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

# df = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")
df = submission.copy()

public_id = list(df['sig_id'].values)

# df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_id = list(df_test['sig_id'].values)

private_id = list(set(test_id)-set(public_id))

df_submit = pd.DataFrame(index = public_id+private_id, columns=TARGET_COL)
df_submit.index.name = 'sig_id'
df_submit[:] = 0
df_predict = BLEND.copy()
df_submit.loc[df_predict.sig_id,:] = df_predict[TARGET_COL].values
df_submit.loc[df_test[df_test.cp_type =='ctl_vehicle'].sig_id] = 0
df_submit.to_csv('submission.csv',index=True)

In [ ]:
print(df_submit.shape)

In [ ]:
df_submit.head()